# Aims to merge initial, partiel, flux (new + evt)

In [13]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
inpi_nature = 'PP' #* nature → ACTES/COMPTES/ETS/etc

import_path = 'SIRETISATION'
match_type='matche'
import_full_path =  "{}/{}".format(import_path,match_type)

initial_file_name = "{}_{}_{}_{}.{}".format('insee',inpi_nature,'initial',match_type,'gz')#insee_PP_initial_matche.gz
new_file_name = "{}_{}_{}_{}.{}".format('insee',inpi_nature,'new',match_type,'gz')#insee_PP_new_matche.gz
initial_filepath = "{}/{}/{}".format(import_path,match_type,initial_file_name)
new_filepath = "{}/{}/{}".format(import_path,match_type,new_file_name)

In [15]:
new_filepath

'SIRETISATION/matche/insee_PP_new_matche.gz'

In [16]:
# instanciate AWS connection
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'
AWS_connection = aws.aws_instantiate(instance_aws, bucket)
#AWS_connection.uploadFileBucket(pathfile =inpi_import_json_filepath)

In [17]:
# get files

pp_initial = AWS_connection.url_instance_bucket(path_file = initial_filepath)
pp_new = AWS_connection.url_instance_bucket(path_file = new_filepath)

In [18]:
new_filepath

'SIRETISATION/matche/insee_PP_new_matche.gz'

In [19]:
pp_new

'https://calfdata.s3.eu-west-3.amazonaws.com/SIRETISATION/matche/insee_PP_new_matche.gz'

In [22]:
## INPI
dtypes__ = {'Siren':'object',
            'Adresse_Ligne1': 'object',
       'Code_Commune': 'object',
       'Code_Postal': 'object',
       'Conjoint_Collab_Pseudo': 'object',
       'DAP_Adresse_Ligne1': 'object',
       'DAP_Code_Commune': 'object',
       'Date_Immatriculation': 'object',
       'Date_1re_Immatriculation': 'object',
       'Date_Radiation': 'object',
       'Date_Greffe': 'object',
       'Sans_Activité': 'object'}

data_pp_new= dd.read_csv(pp_new,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
data_pp_new.head()

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url
0,314488321,31448832100014,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321
1,314488321,31448832100022,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321
2,314488321,31448832100030,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321
3,338065873,33806587300014,O,2017-12-29,NaN,NaN,NaN,NaN,31820,31417,NaN,NaN,NaN,2017-12-29,both,https://data.inpi.fr/entreprises/338065873
4,338065873,33806587300022,O,2017-12-29,NaN,NaN,NaN,NaN,31820,31417,NaN,NaN,NaN,2017-12-29,both,https://data.inpi.fr/entreprises/338065873


In [23]:
data_pp_initial= dd.read_csv(pp_initial,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
data_pp_initial.head()

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url
0,5741509,574150900011,O,1957-06-01,NaN,NaN,NaN,NaN,04270,NaN,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/005741509
1,5915533,591553300011,O,1959-12-28,NaN,NaN,NaN,NaN,80220,80373,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/005915533
2,6472419,647241900032,O,1964-05-27,NaN,NaN,NaN,9 rue de Conan,44780,44098,NaN,NaN,NaN,2017-05-04,both,https://data.inpi.fr/entreprises/006472419
3,6641104,664110400010,O,1966-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-06,both,https://data.inpi.fr/entreprises/006641104
4,7241532,724153200014,O,1980-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-04,both,https://data.inpi.fr/entreprises/007241532


In [25]:
# Union all data
data_pp_all= pd.concat([data_pp_new, data_pp_initial])
data_pp_all=data_pp_new.append()


TypeError: cannot concatenate object of type "<class 'dask.dataframe.core.DataFrame'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid